In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import numpy as np
import os
import current_model as model

# Required fields
rates = [
    [0, 'a1*exp(V/k1)', 0, 0, 0, 0],
    ['b1*exp(-V/k1)', 0, 'a2*exp(V/k2)', 0, 0, 0],
    [0, 'b2*exp(-V/k2)', 0, 'a3*exp(V/k3)', 0, 0],
    [0, 0, 'b3*exp(-V/k3)', 0, 'a4*exp(V/k4)', 0],
    [0, 0, 0, 'b4*exp(-V/k4)', 0, 'a'],
    [0, 0, 0, 0, 'b', 0]
]
# P/Q model:
parameters = {
    'a1' : {'value': 5.89, 'units': '1/ms'},
    'a2' : {'value': 9.21, 'units': '1/ms'},
    'a3' : {'value': 5.20, 'units': '1/ms'},
    'a4' : {'value': 1823.18, 'units': '1/ms'},
    'a' : {'value': 247.71, 'units': '1/ms'},
    'b1' : {'value': 14.99, 'units': '1/ms'},
    'b2' : {'value': 6.63, 'units': '1/ms'},
    'b3' : {'value': 132.80, 'units': '1/ms'},
    'b4' : {'value': 248.58, 'units': '1/ms'},
    'b' : {'value': 8.28, 'units': '1/ms'},
    'k1' : {'value': 62.61, 'units': 'mV'},
    'k2' : {'value': 33.92, 'units': 'mV'},
    'k3' : {'value': 135.08, 'units': 'mV'},
    'k4' : {'value': 20.86, 'units': 'mV'}
}
initial_condition = [1, 0, 0, 0, 0, 0]

# Add optional metadata
name = 'CaV2.1'
state_names = ['C0','C1','C2','C3','C4','O']
conductive = [0, 0, 0, 0, 0, 1]
stimuli = {
    'V' : {
        'time' : [],
        'stim' : [],
        'time_units' : 'ms',
        'stim_units' : 'mV'
    }
}
reference = 'L. Li, J. Bischofberger, and P. Jonas, “Differential Gating and Recruitment of P/Q-, N-, and R-Type Ca2+ Channels in Hippocampal Mossy Fiber Boutons,” en, J. Neurosci., vol. 27, no. 49, pp. 13 420–13 429, Dec. 2007, issn: 0270-6474, 1529-2401.'
doi = 'https://doi.org/10.1523/JNEUROSCI.1709-07.2007'

# Create dictionary of data and metadata
data = {\
    'name' : name,
    'rates': rates,
    'parameters' : parameters,
    'initial_condition': initial_condition,
    'conductive' : conductive,
    'stimuli' : stimuli,
    'state_names': state_names,
    'reference': reference,
    'doi': doi
    }

# Save as JSON
filename = os.path.join('models',data['name']+'.json')
model.export_model(data, filename)

In [9]:
import numpy as np
import os
import current_model as model
import pandas as pd
import plotly.express as px

In [10]:
filename = os.path.join('models','Cav2.1'+'.json')
PQ_model = model.import_model(filename)
print(PQ_model)

CaV2.1: 6-state Markov chain


In [11]:
t_end = 1 # ms
n_points = 1000
time = np.linspace(0, t_end, n_points)

# Constant timeseries
voltage_constant = [50] * len(time)
stim_constant = {
    'V' : {
        'time' : time,
        'stim' : voltage_constant,
        'time_units' : 'ms',
        'stim_units' : 'mV'
    }
}

# Linear ramp
voltage_ramp = 100 * time
stim_ramp = {
    'V' : {
        'time' : time,
        'stim' : voltage_ramp,
        'time_units' : 'ms',
        'stim_units' : 'mV'
    }
}

# AP-like trace
ap_df = pd.read_csv(os.path.join('stimuli', 'AP_voltage.csv'))
ap_time = list(ap_df['Time (ms)'])
ap_stim = list(ap_df['Voltage (mV)'])
stim_ap = {
    'V' : {
        'time' : ap_time,
        'stim' : ap_stim,
        'time_units' : 'ms',
        'stim_units' : 'mV'
    }
}

simulations = [stim_constant, stim_ramp, stim_ap]
PQ_model.run_simulations(simulations)
print('Simulations Complete.')

Simulations Complete.


In [12]:
titles = ['Constant 50 mV', '0-100 mV Ramp', 'AP-Like Waveform']
for sim_idx, simulation in enumerate(PQ_model.simulations):
    time = simulation['stimuli']['V']['time']
    time_label = simulation['stimuli']['V']['time_units']
    columns = [time_label] + PQ_model.metadata['state_names'][:]
    data = np.column_stack((time, simulation['states']))
    results = pd.DataFrame(data, columns=columns)
    fig = px.line(results, x=time_label, y=PQ_model.metadata['state_names'])
    fig.update_layout(title=titles[sim_idx], hovermode='x', template='plotly_dark')
    fig.update_xaxes(ticks='outside', linecolor='black')
    fig.update_yaxes(title='Probability', range=[0,1], ticks='outside', linecolor='black')
    fig.show()